In [7]:
import numpy as np
import pandas as pd
from mip import Model, xsum, minimize, BINARY

In [9]:
df = pd.read_csv("Data.csv")
df

,Place index,Place Name,Latitude,Longitude,Neighbors_indice,Neighbors_weight,time,distance,traffic
0,1,Enghelab Square,35.701146,51.391189,"2,3","1,3","2,7","0.9,3.2","1,1"
1,2,UT Enghelab campus,35.702588,51.395695,"1,4","1,4","2,7","0.95,3.5","1,1"
2,3,Darvazeh Dowlat Metro Station,35.702173,51.425740,"1,5","5,6","7,11","4.1,6","1,0"
3,4,UT-Fanni campus,35.724721,51.388642,"2,5,6","5,8,5","8,10,5","3.9,7.8,5","1,0,0"
4,5,Kasra Hospital,35.739570,51.415285,"3,4,7","6,6,6","10,10,10","6.3,5.4,6.6","0,1,0"
5,6,Baqiyatallah Hospital,35.757897,51.395250,"4,7","4,7","8,10","4.5,6.2","0,1"
6,7,Mofid Children's Hospital,35.760837,51.445851,"5,6,8","6,10,5","7,8,8","5.8,7.8,4.6","1,1,1"
7,8,Gholhak Subway Station,35.773169,51.437945,"7,10,9","3,5,9","3,7,13","1.7,5.1,8.6","1,1,0"
8,9,Tehran International Exhibition Center,35.792741,51.402764,8,7,10,7.4,0
9,10,Ekhtiarie Square,35.787212,51.466496,8,4,9,4,0


In [10]:
n = len(df)
weights = np.zeros((n, n))
for i in range(n):
    Neighbors = df['Neighbors_indice'][i].split(",")
    Neighbors = [int(x) for x in Neighbors]
    w = df['Neighbors_weight'][i].split(",")
    w = [int(x) for x in w]
    for idx, nei in enumerate(Neighbors):
      weights[i][nei-1] = w[idx]

In [11]:
weights

array([[ 0.,  1.,  3.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 1.,  0.,  0.,  4.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 5.,  0.,  0.,  0.,  6.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  5.,  0.,  0.,  8.,  5.,  0.,  0.,  0.,  0.],
       [ 0.,  0.,  6.,  6.,  0.,  0.,  6.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  4.,  0.,  0.,  7.,  0.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  6., 10.,  0.,  5.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  3.,  0.,  9.,  5.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  7.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  4.,  0.,  0.]])

In [73]:
def get_input(df):
    print("The names of the places are:")
    print("1: Enghelab Square , 2: UT Enghelab campus , 3: Darvazeh Dowlat Metro Station")
    print("4: UT-Fanni campus , 5: Kasra Hospital , 6: Baqiyatallah Hospital")
    print("7: Mofid Children's Hospital, 8: Gholhak Subway Station")
    print("9: Tehran International Exhibition Center, 10: Ekhtiarie Square")
    Starting_Point = input("\nPlease enter the Starting Point name just as same as above names : ")
    Destination_Point = input("\nPlease enter the Destination Point name just as same as above names : ")
    
    start = df["Place Name"].str.find(Starting_Point)
    start = list(start)
    start = start.index(0)

    destination = df["Place Name"].str.find(Destination_Point)
    destination = list(destination)
    destination = destination.index(0)

    print("\nIndex of Starting Point is: %d, and it's coordinate is: %f, %f" %(start + 1, df["Latitude"][start], df["Longitude"][start]))
    print("\nIndex of Destination Point is: %d, and it's coordinate is: %f, %f" %(destination + 1, df["Latitude"][destination], df["Longitude"][destination]))
    return start, destination

In [74]:
def define_constants(model, V, weights):
    for i in V:
      for j in V:
        if weights[i][j] == 0:
          model += x[i][j] == 0

    for i in V:
        if i == start:
          value = 1
        elif i == destination:
          value = -1
        else:
          value = 0
        model += xsum(x[i][j] for j in V) - xsum(x[j][i] for j in V) == value

In [81]:
def print_result(x, weights):
    COST = 0
    place = start
    print("\nOptimal Path is:")
    j = 0
    while place != destination:
      if x[place][j].x == 1:
        COST += weights[place][j]
        print(str(place + 1) + " --> ", end='')
        place = j
        j = 0
      else:
        j += 1
    print(str(destination + 1))
    print("\nThe total cost of the optimal rout is:", COST)

In [80]:
start, destination = get_input(df)
V = set(range(n))
model = Model()
x = [[model.add_var(var_type=BINARY) for j in V] for i in V]
model.objective = minimize(xsum(weights[i][j]*x[i][j] for i in V for j in V))
define_constants(model, V, weights)
model.optimize()
print_result(x, weights)

The names of the places are:
1: Enghelab Square , 2: UT Enghelab campus , 3: Darvazeh Dowlat Metro Station
4: UT-Fanni campus , 5: Kasra Hospital , 6: Baqiyatallah Hospital
7: Mofid Children's Hospital, 8: Gholhak Subway Station
9: Tehran International Exhibition Center, 10: Ekhtiarie Square

Please enter the Starting Point name just as same as above names : Ekhtiarie Square

Please enter the Destination Point name just as same as above names : UT-Fanni campus

Index of Starting Point is: 10, and it's coordinate is: 35.787212, 51.466496

Index of Destination Point is: 4, and it's coordinate is: 35.724721, 51.388642

Optimal Path is:
10 --> 8 --> 7 --> 5 --> 4

The total cost of the optimal rout is: 19.0
